# Introduction

Until now we have worked with one table at a time. However, there are many scenarios in which the information in a single table is not sufficient to answer our questions.

We have put stars and planets in separate tables. Apart from being conceptually different, they have different physical properties.

Separate tables also eliminates redundancy. If the star data was copied for each planet, every update to a star's measurements may require multiple rows in the Planet table to be updated.

But, if we want to study the entire planetary system surrounding a star, we need both tables.

We can use SQL's join operations to combine two or more tables in our queries to get all the information we want.

# Joining Tables

There are several ways to join data from multiple tables in SQL, the simplest one is called an equi-join. An equi-join usually tests for equality on attributes shared across tables.

For example, our Star and Planet tables share the kepler_id attribute, which links planets to their respective stars.

Using this column, we can join the data in both tables in two steps:

1. List both tables in the FROM clause
2. Require the common attribute to match in a WHERE condition

In [ ]:
SELECT Star.kepler_id, Planet.koi_name
FROM Star, Planet
WHERE Star.kepler_id = Planet.kepler_id;

The query above returns the **kepler_id** and the planet's unique **koi_name**. For each **kepler_id**, there may be more than one **koi_name**, meaning that the planets share the same star.

# Note: Old school style!
This SQL join syntax doesn't make the join columns obvious, and while it is still commonly used, it is no longer preferred.

# Ambiguity and conditions

Let's have a closer look at the join query from the previous slide:

In [ ]:
SELECT Star.kepler_id, Planet.koi_name
FROM Star, Planet
WHERE Star.kepler_id = Planet.kepler_id;

Two points that stand out here are the explicit use of table names and the condition comparing the common attribute.

When we have tables that share attributes of the same name, we need to specify which table we're talking about to avoid ambuguity. Take a look at what happens if we don't:

In [ ]:
SELECT kepler_id, koi_name
FROM Star, Planet
WHERE kepler_id = kepler_id;

In [ ]:
psql:query.sql:3: ERROR:  column reference "kepler_id" is ambiguous
LINE 1: SELECT kepler_id, koi_name
               ^

The query now returns an error due to the ambiguity problem described above. How is the SQL server supposed to know which kepler_id we mean?

If we're joining tables with long names, it is handy to introduce shorter aliases for the table names with AS, like this:

In [ ]:
SELECT s.kepler_id, p.koi_name
FROM Star AS s, Planet AS p
WHERE s.kepler_id = p.kepler_id

This makes our query a lot shorter and helps with the readability as well. You can also use aliases for the attributes themselves to change the returned column name.

# Missing join conditions

The condition in the WHERE clause is also called a join condition, it specifies how the data is to be joined. If we don't include such a condition, we get the following result:

In [ ]:
SELECT Star.kepler_id, Planet.koi_name
FROM Star, Planet

The return of this query is quite different from the result we get with the equality condition. By taking this condition out, we've transformed our equi-join into a *cross*- or *cartesian-join*.

Every row in Star is combined with each row in Planet, and hence the resulting table has (number of stars)x(number of planets) rows. Check for yourself:

In [ ]:
SELECT COUNT(*) FROM Star;
SELECT COUNT(*) FROM Planet;

If you don't want a cross-join, remember to include a join condition in your query.

# **Question: Systems with small planets**

Write a query that returns the radius of each star and planet pair whose radii have a ratio greater than the Sun-to-Earth radius ratio. Order the results in descending order based on the stellar radii. Use sun_radius and planet_radius as attribute aliases for the star and planet radii.

For this problem you will have to join the two tables to find all planets belonging to a given star and use a condition to select those results which fulfill the size requirement above.

The result should look like this on the example data:

| kepler_id | count |
|-----------|-------|
| 4139816   | 4     |
| 8395660   | 4     |
| 10910878  | 3     |
| 10872983  | 3     |
| 5358241   | 3     |
| 10601284  | 3     |
| 3832474   | 3     |
| 11754553  | 3     |
| 9579641   | 3     |
| 12366084  | 2     |
| 11018648  | 2     |
| 11918099  | 2     |
| 12066335  | 2     |
| 10187017  | 2     |
| 10797460  | 2     |
| 3342970   | 2     |
| 12470844  | 2     |
| 5456651   | 2     |


# **Hints**:
1. The units of star and planet radii in our tables are relative to our Sun and Earth radii respectively;
2. Planets orbiting a given star share its **kepler_id** attribute.

# ⌛Solution:

In [ ]:
SELECT Star.radius AS sun_radius, Planet.radius AS planet_radius
FROM Star, Planet
WHERE Star.kepler_id = Planet.kepler_id
AND Star.radius / Planet.radius > 1
ORDER BY sun_radius DESC

# Inner vs. Outer joins

The types of joins that we've looked at so far fall into the category of inner joins, in which data is combined only when the specified condition is true. The inner join is implicit when calling the JOIN operator; we might have just as well written INNER JOIN in all of our previous queries.

The complement to inner joins are so-called outer joins - here table rows can be combined even if there is no match found to fulfill the join condition. We distinguish between three types of outer joins:


$\text{<table1>}$ LEFT OUTER JOIN $\text{<table2>}$

Here all rows from $\text{<table1>}$ are kept and missing matches from $\text{<table2>}$ are replaced with NULL values.

$\text{<table1>}$ RIGHT OUTER JOIN $\text{<table2>}$

All rows from $\text{<table2>}$ are kept and missing matches from $\text{<table1>}$ are replaced with NULL values.

$\text{<table1>}$ FULL OUTER JOIN $\text{<table2>}$

All rows from both tables are kept.

# Left and Right Outer join

We can use the outer join just like we used the inner joins with the two keywords USING and ON.

For example, to create an outer join that keeps all rows in the Star table whether or not there are matching planets for this star, we would write:

In [ ]:
SELECT S.kepler_id, P.koi_name
FROM Star S
LEFT OUTER JOIN Planet P USING(kepler_id);

As you scroll down you can see that there are a couple of rows at the end which are filled with NULL values, since there are no planets in the Planet table for these **kepler_ids**.

In the same spirit, we could use a RIGHT OUTER JOIN to keep all entries in the Planet table on the right-hand side:

In [ ]:
SELECT S.kepler_id, P.koi_name
FROM Star S
RIGHT OUTER JOIN Planet P USING(kepler_id);

# **Note**
The following two joins yield the same result:

$\text{<table1>}$ LEFT OUTER JOIN $\text{<table2>}$

$\text{<table2>}$ RIGHT OUTER JOIN $\text{<table1>}$

It's just about on which side of the join operator the tables are.

# Full Outer join

Left and right outer joins allow us to keep all values in the table on the left- or right-hand side of the JOIN operator, by replacing missing values in the other table with NULL values. If we want to keep the full data in both tables, we can use the FULL OUTER JOIN:

In [ ]:
SELECT S.kepler_id, P.koi_name
FROM Star S
FULL OUTER JOIN Planet P USING(kepler_id);

Here you can see that we have kept the full data of both tables and missing values are again shown as empty fields in both columns.

Note that you always need to specify which type of join to perform, i.e. LEFT, RIGHT or FULL. The following does not work:

In [ ]:
SELECT *
FROM Star OUTER JOIN Planet USING(kepler_id);

psql:query.sql:2: ERROR:  syntax error at or near "OUTER"

LINE 2:  FROM Star OUTER JOIN Planet USING(kepler_id);

                   ^

# **Question: Lonely stars**

To practise your outer joins, write a query which returns the **kepler_id**, **t_eff** and **radius** for all stars in the Star table which haven't got a planet as join partner. Order the resulting table based on the **t_eff** attribute in descending order.

The result should look like this on the example data:

| kepler_id | t_eff | radius |
|-----------|-------|--------|
| 6922244   | 6225  | 1.451  |
| 10480982  | 6117  | 0.947  |
| 10797460  | 5850  | 1.04   |
| 6862328   | 5796  | 0.871  |
| 3246984   | 5735  | 0.973  |
| 3114167   | 5666  | 0.677  |
| 11414511  | 5653  | 0.965  |
| 4275191   | 5557  | 0.781  |
| 3832474   | 5485  | 0.867  |

# **Note**:
1. a LEFT OUTER JOIN on Star with Planet;
2. an IS NULL check on Planet.koi_name to look for stars without planets;
3. an ORDER BY to sort by descending effective temperature.

# ⌛Solution:

In [ ]:
SELECT S.kepler_id, S.t_eff, S.radius
FROM Star AS S
LEFT OUTER JOIN Planet as P USING (kepler_id)
WHERE P.koi_name IS NULL
ORDER BY S.t_eff DESC;

# Advanced queries

With joins on hand as well, we've built up a solid foundation of querying skills. The last thing we're going to look at in this module is how we can make our queries a bit more arbitrary. But before we do that, let's have a look at why we would be interested in that.

**Example 1**: How can we find all stars which are larger than the average star in our table? Naively, we might try:

In [ ]:
SELECT * FROM Star
WHERE Star.radius > AVG(Star.radius)

psql:query.sql:2: ERROR:  aggregate functions are not allowed in WHERE

LINE 2:  WHERE Star.radius > AVG(Star.radius)

                             ^

But this doesn't work, because we can't use aggregate functions like AVG inside a WHERE clause.

We could write two queries, one for calculating the average and one where we take the calculated average and use it as a lower bound, but that doesn't work in general anymore. Any time the table is changed the average may change.

**Example 2**: How can we find out how many rows would be returned by a cross-join of two or more tables (by calculating the product of the table entries)? Can you think of a way? The following doesn't really work:

In [ ]:
SELECT COUNT(Star.*) * COUNT(Planet.*)
FROM Star JOIN Planet USING(kepler_id);

SELECT COUNT(*) FROM Star;
SELECT COUNT(*) FROM Planet;

As you can tell from looking at the individual counts. But then again, we'd like to have something arbitrary, not a two-step solution.

# Subqueries to the rescue

One thing that the two problems had in common was that we actually needed two or more queries to answer the question, but we didn't want to execute them one by one. For these types of problems we can use nested queries or subqueries.

The idea is that we nest additional queries in our main query to return the result we can't access within a single query. For example, to calculate the average on the fly we could write:

In [ ]:
SELECT * FROM Star
WHERE Star.radius > (
  SELECT AVG(radius) FROM Star
);

Here we nested the query to calculate the average within the WHERE clause using parentheses.

Subqueries can be nested into most clauses, including the SELECT clause. For example, to solve our second problem, we would like to calculate a simple product in this clause like:

In [ ]:
SELECT 2 * 3;

Where the factors in our product would be the individual row counts. Using subqueries, we can write:

In [ ]:
SELECT
  (SELECT COUNT(*) FROM Star) *
  (SELECT COUNT(*) FROM Planet);

# Co- vs. non-co-related subqueries

We distinguish between two types of subqueries:

* **Co-related**: the subquery is executed for each element of the outer query.
* **Non-co-related**: the subquery is executed only once.
It is useful to recognise these two different types as they can vastly differ in their performance. Let's look at two examples.

In [ ]:
SELECT s.kepler_id
FROM Star s
WHERE EXISTS (
  SELECT * FROM Planet p
  WHERE s.kepler_id = p.kepler_id
    AND p.radius < 1
);

This is a co-related subquery. We select each row in Star where the corresponding kepler_id in Planet is a planet with radius smaller than 1. We may have to query the Planet table for each row in Star and check if the result is not empty using EXISTS.

We could also solve this problem in a non-co-related subquery:

In [ ]:
SELECT s.kepler_id
FROM Star s
WHERE s.kepler_id IN (
  SELECT p.kepler_id FROM Planet p
  WHERE p.radius < 1
);

Here we generate a table with the **kepler_ids** of all planets with a radius smaller than one, and then we check for each star if its **kepler_id** is contained in the returned Planet table. We query the Planet table only once.

# ⏰**Identify co- and non-co-related queries**

If an attribute in the encapsulating query is used in the nested query, it's going to be a co-related subquery.

# Subqueries vs. joins

On the last slide we were performing a join query, since we've used both the Star and the Planet table. Joins are in fact something that subqueries are frequently used for.

So is there a preferred way? If you want to choose between different solutions to the same problem, you could for example compare the execution times of the different approaches.

PostgreSQL offers a particularly simple solution for timing queries: all you have to do is to add \timing at the beginning of your script. The following code snippet compares both the subquery and explicit join query and times their execution:

In [ ]:
\timing
-- Join with subqueries
SELECT s.kepler_id
FROM Star s
WHERE s.kepler_id IN (
  SELECT p.kepler_id FROM Planet p
  WHERE p.radius < 1
);

-- Join with JOIN operator
SELECT DISTINCT(s.kepler_id)
FROM Star s
JOIN Planet p USING (kepler_id)
WHERE p.radius < 1;

In this example, the explicit join query is a lot faster than the subquery join.

# **Question: Subquery joint stars and planets**

Write a query which queries both the Star and the Planet table and calculates the following quantities:

1. the average value of the planets' equilibrium temperature **t_eq**, rounded to one decimal place;
2. the minimum effective temperature **t_eff** of the stars;
3. the maximum value of **t_eff**;

Your query should only use those star-planet pairs whose stars have a higher temperature (**t_eff**) than the average star temperature in the table. Try to use a subquery to solve this problem!

# ⌛Solution:

In [ ]:
WITH StarAverageTemp AS (
    SELECT AVG(t_eff) AS avg_t_eff
    FROM Star
)
SELECT
    ROUND(AVG(Planet.t_eq), 1) AS round,
    MIN(Star.t_eff) AS min,
    MAX(Star.t_eff) AS max
FROM Star
JOIN Planet ON Star.kepler_id = Planet.kepler_id
WHERE Star.t_eff > (SELECT avg_t_eff FROM StarAverageTemp);

# **Question: Correlated sizes?**

Write a query which finds the radii of those planets in the Planet table which orbit the five largest stars in the Star table.

Your query should return the planet's koi_name and radius as well as the corresponding star radius.

You should expect to see the following star radii in your result:



In [ ]:
SELECT radius FROM Star
ORDER BY radius DESC
LIMIT 5;

| radius |
|--------|
| 1.991  |
| 1.451  |
| 1.411  |
| 1.332  |
| 1.103  |


# Hint:

this problem uses (nearly) everything!

This problem needs a subquery (to find the largest 5 stars by radius) and lots of other things you've learned this week! It can be done with, or without, a join though.

# ⌛Solution:

In [ ]:
WITH TopFiveLargestStars AS (
    SELECT kepler_id, radius AS star_radius
    FROM Star
    ORDER BY radius DESC
    LIMIT 5
)
SELECT
    Planet.koi_name,
    Planet.radius AS radius,
    TopFiveLargestStars.star_radius AS radius
FROM Planet
JOIN TopFiveLargestStars ON Planet.kepler_id = TopFiveLargestStars.kepler_id;